In [2]:
###6.2.1 互相关运算
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):  #@save
    """计算二维互相关运算"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [4]:
###6.2.2 卷积层
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [5]:
###6.2.3 图像中物体边缘检测
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [6]:
K = torch.tensor([[1.0, -1.0]])

In [7]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [8]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [9]:
###6.2.4 通过数据学习核数组
# 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
conv2d = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # 学习率

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 11.338
epoch 4, loss 3.451
epoch 6, loss 1.213
epoch 8, loss 0.463
epoch 10, loss 0.184


In [10]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9446, -1.0323]])

###  6.2.6. 特征映射和感受野
如在 6.1.4.1节中所述， 图6.2.1中输出的卷积层有时被称为特征映射（feature map），因为它可以被视为一个输入映射到下一层的空间维度的转换器。 在卷积神经网络中，对于某一层的任意元素$x$，其感受野（receptive field）是指在前向传播期间可能影响$x$计算的所有元素（来自所有先前层）。

1. 构建一个具有对角线边缘的图像X。
    1. 如果将本节中举例的卷积核K应用于X，会发生什么情况?
        - 卷积核K会检测到图像中的水平边缘，输出的结果会在边缘处有较大的值。
    2. 如果转置X会发生什么?
        - 转置X会将图像的行和列互换，卷积核K应用于转置后的X会检测到垂直边缘。
    3. 如果转置K会发生什么?
        - 转置K会将卷积核的行和列互换，变成检测垂直边缘的卷积核，应用于原始图像X会检测到垂直边缘。
2. 在我们创建的Conv2D自动求导时，有什么错误消息?
    - 如果卷积核的尺寸不匹配输入张量的尺寸，可能会出现尺寸不匹配的错误。
3. 如何通过改变输入张量和卷积核张量，将互相关运算表示为矩阵乘法?
    - 可以将输入张量和卷积核张量展平为向量，然后使用矩阵乘法来表示互相关运算。
4. 手工设计一些卷积核。
    1. 二阶导数的核的形式是什么?
        - 二阶导数的卷积核可以表示为：tensor([[1, -2, 1]]) 或 tensor([[1], [-2], [1]])。
    2. 积分的核的形式是什么?
        - 积分的卷积核可以表示为：tensor([[1, 1, 1]]) 或 tensor([[1], [1], [1]])。
    3. 得到d次导数的最小核的大小是多少?
        - 得到d次导数的最小卷积核的大小为d+1。
